# Imports

In [1]:
import requests
import bs4
import webbrowser

# Requesting from law.justia.com

In [ ]:
#
penal_res = requests.get('https://law.justia.com/codes/new-york/2018/pen/part-3/title-g/article-100/100.00/')
penal_res.raise_for_status()
penal_res.status_code
#

# Practice Souping

In [ ]:
soup = bs4.BeautifulSoup(penal_res.text, 'html.parser')

In [ ]:
type(soup)

In [ ]:
code_content = soup.select('p')

In [ ]:
type(code_content), len(code_content)

In [ ]:
for i in range(0,2):
    lines = code_content[i].getText().replace("\r", "").replace("\n", "").replace("  ", " ")
    
    print(lines+'\n')

In [ ]:
line = code_content[0].getText()

In [ ]:
line

# Automated Penal Code Webpage Scrape

In [2]:
def get_code_contents_from_url(url):
    res = requests.get(url)
    res.raise_for_status()
    soup = bs4.BeautifulSoup(res.text, 'html.parser')
    content_paragraphs = soup.select('p')
    code_text = ""
    for i in range(len(content_paragraphs)):
        paragraph = content_paragraphs[i].getText().replace("\r", "").replace("\n", "").replace("  ", " ").lower()
        code_text += paragraph + " "
    
    return code_text

In [ ]:
text = get_code_from_page('https://law.justia.com/codes/new-york/2018/pen/part-3/title-g/article-100/100.00/')
print(text)

In [ ]:
text = get_code_from_page('https://law.justia.com/codes/new-york/2018/pen/part-3/title-k/article-170/170.00/')
print(text)

In [ ]:
## OK it works :)

# Going Through a Site Practice

In [ ]:
res = requests.get('https://law.justia.com/codes/new-york/2018/pen/part-3/title-k/')
res.raise_for_status()

In [ ]:
soup = bs4.BeautifulSoup(res.text, 'html.parser')
list = soup.select('.codes-listing')
type(list), len(list)

In [ ]:
deep_soup = bs4.BeautifulSoup(str(list[0]), 'html.parser')
links = deep_soup.select('a')
type(links), len(links)

In [ ]:
for i in range(0, len(links)):
    print(links[i].get('href'))

# Automated Link Scraper

In [3]:
def get_links_from_url(url):
    res = requests.get(url)
    res.raise_for_status()
    soup = bs4.BeautifulSoup(res.text, 'html.parser')
    code_listing = soup.select('.codes-listing')
    if len(code_listing) == 0:
        print('found base page')
        return 0
    deep_soup = bs4.BeautifulSoup(str(code_listing[0]), 'html.parser')
    links = deep_soup.select('a')
    
    print('found ' + str(len(links)) + ' links')
    
    return links

In [ ]:
tag_array = get_links_from_url('https://law.justia.com/codes/new-york/2018/pen/part-3/')

# Automated Link Diver

In [4]:
def get_page(tag):
        newUrl = 'https://law.justia.com' + tag.get('href')
        res = requests.get(newUrl)
        res.raise_for_status()
        
        return res.text

# Link Scraper + Diver

### but first... Automated Link Checker

In [5]:
def has_links(page_text):
    soup = bs4.BeautifulSoup(res.text, 'html.parser')
    code_listing = soup.select('.codes-listing')
    if len(code_listing) > 0:
        return True
    else:
        return False

In [6]:
import random
import uuid

def get_justia_penal_codes():
    need_column_labels = True
    outfile = 'penal_codes.csv'
    outsize = 1024 * 1024 * 1024 # 1GB
    size = 0
    with open(outfile, 'ab') as csvfile:
        if need_column_labels:
            csvfile.write(bytes('id, citation\n', 'utf-8'))
            need_column_labels = False
    url = 'https://law.justia.com/codes/new-york/2018/pen/part-3/'
    titles = get_links_from_url(url)
    
    
    # crawl the entire website
    for i,title in enumerate(titles):
        titleUrl = 'https://law.justia.com'+title.get('href')
        articles = get_links_from_url(titleUrl)
        
        if size < outsize:
            
            for j,article in enumerate(articles):
                articleUrl = 'https://law.justia.com'+article.get('href')
                citations = get_links_from_url(articleUrl)
                
                if size < outsize:
                    
                    with open(outfile, 'ab') as csvfile:
                        
                        for k,citation in enumerate(citations):
                            id_text = 'Title ' + str(i) + ' Article ' + str(j) + ' Citation ' + str(k)
                            
                            citationUrl = 'https://law.justia.com'+citation.get('href')
                            txt = '%s,%s\n' % (id_text, get_code_contents_from_url(citationUrl).replace(",", ""))
                            size += len(txt)
                            csvfile.write(bytes(txt, 'utf-8'))
                else:
                    break;
        else:
            break;
            
    print('done')

In [7]:
get_justia_penal_codes()

found 10 links
found 4 links
found 7 links
found 10 links
found 3 links
found 7 links
found 5 links
found 30 links
found 4 links
found 14 links
found 28 links
found 18 links
found 3 links
found 10 links
found 21 links
found 6 links
found 5 links
found 12 links
found 12 links
found 11 links
found 4 links
found 28 links
found 10 links
found 19 links
found 10 links
found 15 links
found 7 links
found 7 links
found 5 links
found 20 links
found 4 links
found 7 links
found 35 links
found 5 links
found 14 links
found 19 links
found 15 links
found 11 links
found 35 links
found 5 links
found 31 links
found 12 links
found 19 links
found 25 links
found 11 links
found 5 links
found 34 links
found 2 links
found 4 links
found 7 links
found 14 links
found 3 links
found 10 links
found 16 links
found 9 links
found 3 links
found 29 links
found 8 links
found 13 links
done


In [52]:
print(test_scrap)

*Title 0 Article 0 Citation 0*

a person is guilty of criminal solicitation in the fifth degree when, with intent that another person engage in conduct constituting a crime, he solicits, requests, commands, importunes or otherwise attempts to cause such other person to engage in such conduct. criminal solicitation in the fifth degree is a violation. ask a lawyer find a lawyer                              subscribe to justia's free newsletters featuring summaries of federal and state court opinions.                



*Title 0 Article 0 Citation 1*

a person is guilty of criminal solicitation in the fourth degree when: 1. with intent that another person engage in conduct constituting a felony, he solicits, requests, commands, importunes or  otherwise attempts to cause such other person to engage in such conduct; or 2. being over eighteen years of age, with intent that another person under sixteen years of age engage in conduct that would constitute a crime, he solicits, requests, comman

## Writing to CSV